### ClearML configure

In [9]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=V8AMVYWM2GZ50RRHALQUOQV7GMHHKI
%env CLEARML_API_SECRET_KEY=89uJvM4pCoyT_CZxhYykQKx4fgukV2vh0n7F5yVKYj1Yn-etD6DpHN9oVV0riE2dQWE


env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=V8AMVYWM2GZ50RRHALQUOQV7GMHHKI
env: CLEARML_API_SECRET_KEY=89uJvM4pCoyT_CZxhYykQKx4fgukV2vh0n7F5yVKYj1Yn-etD6DpHN9oVV0riE2dQWE


In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from clearml import Task

# Простая сверточная нейронная сеть

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Сверточная нейронная сеть с добавленным слоем Batch Normalization

In [12]:
class NetBN(nn.Module):  # Новое имя класса для модели с BN
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # <- Исправлено
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x)))) # BN после conv1 и ReLU
        x = self.pool(F.relu(self.bn2(self.conv2(x)))) # BN после conv2 и ReLU
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Подгрузка данных

In [13]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


### Проверка доступа, не обращай внимания, у меня не качался датасет

In [14]:
import os
from torchvision import transforms
import sys

# Создаём директорию для данных
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)

# Проверяем права доступа к директории
print(f"Проверка прав доступа к директории {data_dir}")
print(f"Существует: {os.path.exists(data_dir)}")
print(f"Доступ на запись: {os.access(data_dir, os.W_OK)}")

# Настройка трансформаций
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Попытка загрузки с подробным выводом ошибок
try:
    print("Начинаем загрузку тренировочного набора данных...")
    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, 
        train=True, 
        download=True, 
        transform=transform
    )
    print("Тренировочный набор данных успешно загружен")
    
    print("Начинаем загрузку тестового набора данных...")
    testset = torchvision.datasets.CIFAR10(
        root=data_dir, 
        train=False, 
        download=True, 
        transform=transform
    )
    print("Тестовый набор данных успешно загружен")

except Exception as e:
    print(f"Произошла ошибка при загрузке данных: {e}")
    print(f"Тип ошибки: {type(e).__name__}")
    print("Полный traceback:")
    import traceback
    traceback.print_exc()

# Если загрузка успешна, создаём DataLoader
try:
    trainloader = torch.utils.data.DataLoader(
        trainset, 
        batch_size=64, 
        shuffle=True, 
        num_workers=2
    )
    testloader = torch.utils.data.DataLoader(
        testset, 
        batch_size=1000, 
        shuffle=False, 
        num_workers=2
    )
    print("DataLoader'ы успешно созданы")
except NameError:
    print("DataLoader'ы не созданы, так как возникла ошибка при загрузке данных")
except Exception as e:
    print(f"Ошибка при создании DataLoader: {e}")


Проверка прав доступа к директории ./data
Существует: True
Доступ на запись: True
Начинаем загрузку тренировочного набора данных...
Files already downloaded and verified
Тренировочный набор данных успешно загружен
Начинаем загрузку тестового набора данных...
Files already downloaded and verified
Тестовый набор данных успешно загружен
DataLoader'ы успешно созданы


### Описание train и test

In [17]:
def train(model, optimizer, epochs, trainloader, series='Baseline'):
    criterion = nn.CrossEntropyLoss() # Используем CrossEntropyLoss
    for epoch in range(1, epochs + 1):
        model.train()
        for batch_idx, (data, target) in enumerate(trainloader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        train_accuracy = test(model, testloader, "Train", series, epoch)
            # task.get_logger().report_scalar(title='Accuracy', series=series, value=train_accuracy, iteration=epoch)
        task.get_logger().report_scalar(title='Loss', series=series, value=loss.item(), iteration=epoch)
        print(f'Train Epoch: {epoch}\tLoss: {loss.item():.6f}\tAccuracy: {train_accuracy:.2f}%')


def test(model, dataloader, data_type="Test", series="Baseline", epoch=0):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss(reduction='sum') # Используем CrossEntropyLoss
    with torch.no_grad():
        for data, target in dataloader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(dataloader.dataset)
    accuracy = 100. * correct / len(dataloader.dataset)
    task.get_logger().report_scalar(title=f'{data_type} Loss', series=f'{data_type} {series}', value=test_loss, iteration=epoch)
    print(f'\n{data_type} set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(dataloader.dataset)} ({accuracy:.0f}%)\n')
    return accuracy

### Дефолтный запуск

In [18]:
task = Task.init(project_name="MLOps_Lab", task_name="CIFAR-10_Experiment_Default", )

task.get_logger().report_text('Default Testing')

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
train(model, optimizer, epochs=5, trainloader=trainloader)
test_accuracy = test(model, testloader, series='Baseline')

task.get_logger().report_scalar(title='Test Accuracy', series='Baseline', value=test_accuracy, iteration=0)

task.close()

Default Testing

Train set: Average loss: 1.2125, Accuracy: 5771/10000 (58%)

Train Epoch: 1	Loss: 1.063691	Accuracy: 57.71%

Train set: Average loss: 0.9699, Accuracy: 6583/10000 (66%)

Train Epoch: 2	Loss: 0.877451	Accuracy: 65.83%

Train set: Average loss: 0.8925, Accuracy: 6862/10000 (69%)

Train Epoch: 3	Loss: 0.748714	Accuracy: 68.62%

Train set: Average loss: 0.8322, Accuracy: 7137/10000 (71%)

Train Epoch: 4	Loss: 0.729443	Accuracy: 71.37%

Train set: Average loss: 0.8608, Accuracy: 7205/10000 (72%)

Train Epoch: 5	Loss: 0.520230	Accuracy: 72.05%

Test set: Average loss: 0.8608, Accuracy: 7205/10000 (72%)



### Гипотеза 1: Увеличение количества эпох

In [19]:
task = Task.init(project_name="MLOps_Lab", task_name="CIFAR-10_Experiment_Hypothesis_1")

task.get_logger().report_text('Testing Hypothesis 1')

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
train(model, optimizer, epochs=10, trainloader=trainloader, series='MoreEpoch')
test_accuracy = test(model, testloader, series='MoreEpoch')

task.get_logger().report_scalar(title='Test Accuracy', series='MoreEpoch', value=test_accuracy, iteration=0)

task.close()

Could not read Jupyter Notebook: No module named 'nbconvert'
Please install nbconvert using "pip install nbconvert"


ClearML Task: created new task id=00d7e93f778a4dc99cf066def1cc00e3
ClearML results page: https://app.clear.ml/projects/dbc17164c4b14c1ca302acb9aa34ebd2/experiments/00d7e93f778a4dc99cf066def1cc00e3/output/log
Testing Hypothesis 1
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring

Train set: Average loss: 1.1761, Accuracy: 5879/10000 (59%)

Train Epoch: 1	Loss: 1.050064	Accuracy: 58.79%

Train set: Average loss: 1.0105, Accuracy: 6517/10000 (65%)

Train Epoch: 2	Loss: 0.789073	Accuracy: 65.17%

Train set: Average loss: 0.8870, Accuracy: 7023/10000 (70%)

Train Epoch: 3	Loss: 0.766194	Accuracy: 70.23%

Train set: Average loss: 0.8498, Accuracy: 7151/10000 (72%)

Train Epoch: 4	Loss: 0.673228	Accuracy: 71.51%

Train set: Average loss: 0.8487, Accuracy: 7202/10000 (72%)

Train Epoch: 5	Loss: 0.279211	Accuracy: 72.02%

Train set: Average loss: 0.9629, Accuracy: 7128/10000 (71%)

Train Epoch: 6	Loss: 0.653534	Accuracy: 71.28%

Train set: Average loss: 1.

### Гипотеза 2: Batch Normalization

In [20]:
task = Task.init(project_name="MLOps_Lab", task_name="CIFAR-10_Experiment_Hypothesis_2")

task.get_logger().report_text('Testing Hypothesis 2')

model_bn = NetBN()  # Модель с BN
optimizer_bn = optim.SGD(model_bn.parameters(), lr=0.01, momentum=0.9)
train(model_bn, optimizer_bn, epochs=5, trainloader=trainloader, series='Batch Normalization')
test_accuracy_bn = test(model_bn, testloader, series='Batch Normalization')

task.get_logger().report_scalar(title='Test Accuracy', series='Batch Normalization', value=test_accuracy, iteration=0)

task.close()

Could not read Jupyter Notebook: No module named 'nbconvert'
Please install nbconvert using "pip install nbconvert"


ClearML Task: created new task id=0aebb65c63414295a735dfcd392ee3fc
ClearML results page: https://app.clear.ml/projects/dbc17164c4b14c1ca302acb9aa34ebd2/experiments/0aebb65c63414295a735dfcd392ee3fc/output/log
Testing Hypothesis 2
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring

Train set: Average loss: 1.0101, Accuracy: 6431/10000 (64%)

Train Epoch: 1	Loss: 0.822444	Accuracy: 64.31%

Train set: Average loss: 0.9907, Accuracy: 6648/10000 (66%)

Train Epoch: 2	Loss: 0.600236	Accuracy: 66.48%

Train set: Average loss: 0.8166, Accuracy: 7186/10000 (72%)

Train Epoch: 3	Loss: 0.725312	Accuracy: 71.86%

Train set: Average loss: 0.7718, Accuracy: 7357/10000 (74%)

Train Epoch: 4	Loss: 0.629568	Accuracy: 73.57%

Train set: Average loss: 0.8365, Accuracy: 7255/10000 (73%)

Train Epoch: 5	Loss: 0.439213	Accuracy: 72.55%

Test set: Average loss: 0.8365, Accuracy: 7255/10000 (73%)



### Гипотеза 3: Adam Optimizer

In [22]:
task = Task.init(project_name="MLOps_Lab", task_name="CIFAR-10_Experiment_Hypothesis_3")

task.get_logger().report_text('Testing Hypothesis 3')

model3 = Net()
optimizer3 = optim.Adam(model3.parameters(), lr=0.001)
train(model3, optimizer3, epochs=5, trainloader=trainloader, series='Adam Optimizer')
accuracy3 = test(model3, testloader, series='Adam Optimizer')

task.get_logger().report_scalar(title='Test Accuracy', series='Adam Optimizer', value=accuracy3, iteration=0)

task.close()

Testing Hypothesis 3

Train set: Average loss: 1.0298, Accuracy: 6410/10000 (64%)

Train Epoch: 1	Loss: 0.811755	Accuracy: 64.10%

Train set: Average loss: 0.9522, Accuracy: 6712/10000 (67%)

Train Epoch: 2	Loss: 0.710793	Accuracy: 67.12%

Train set: Average loss: 0.8693, Accuracy: 7024/10000 (70%)

Train Epoch: 3	Loss: 0.541665	Accuracy: 70.24%

Train set: Average loss: 0.8809, Accuracy: 7010/10000 (70%)

Train Epoch: 4	Loss: 0.556091	Accuracy: 70.10%

Train set: Average loss: 0.8471, Accuracy: 7224/10000 (72%)

Train Epoch: 5	Loss: 0.270223	Accuracy: 72.24%

Test set: Average loss: 0.8471, Accuracy: 7224/10000 (72%)



### AllIn


In [23]:
task = Task.init(project_name="MLOps_Lab", task_name="CIFAR-10_Experiment_Final")

task.get_logger().report_text('Testing Hypothesis AllIn')

model3 = NetBN()
optimizer3 = optim.Adam(model3.parameters(), lr=0.001)
train(model3, optimizer3, epochs=10, trainloader=trainloader, series='AllIn')
accuracy3 = test(model3, testloader, series='AllIn')

task.get_logger().report_scalar(title='Test Accuracy', series='AllIn', value=accuracy3, iteration=0)

task.close()

ClearML Task: created new task id=e64013c1d55b40249ccdfb2fa7bfbca5
ClearML results page: https://app.clear.ml/projects/dbc17164c4b14c1ca302acb9aa34ebd2/experiments/e64013c1d55b40249ccdfb2fa7bfbca5/output/log
Testing Hypothesis AllIn
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring

Train set: Average loss: 1.1181, Accuracy: 6141/10000 (61%)

Train Epoch: 1	Loss: 0.709194	Accuracy: 61.41%

Train set: Average loss: 1.1807, Accuracy: 5977/10000 (60%)

Train Epoch: 2	Loss: 0.747219	Accuracy: 59.77%

Train set: Average loss: 0.9300, Accuracy: 6918/10000 (69%)

Train Epoch: 3	Loss: 0.948523	Accuracy: 69.18%

Train set: Average loss: 0.7877, Accuracy: 7234/10000 (72%)

Train Epoch: 4	Loss: 1.053416	Accuracy: 72.34%

Train set: Average loss: 0.8583, Accuracy: 7139/10000 (71%)

Train Epoch: 5	Loss: 0.277816	Accuracy: 71.39%

Train set: Average loss: 0.7928, Accuracy: 7357/10000 (74%)

Train Epoch: 6	Loss: 0.846929	Accuracy: 73.57%

Train set: Average loss

In [24]:
task.close()